https://www.kaggle.com/ruruamour/sample-code2-augmentation  
このサンプルコードでは少し精度が伸びましたがまだまだ判定精度を上げることができます。  
このNotebookでは転移学習という手法を使って簡単に高精度な機械学習モデルを作成します。  

（The above sample code shows a slight increase in accuracy, but the accuracy can still be improved.    
This Notebook uses a technique called transfer learning to easily create a highly accurate machine learning model.  ）

# 調整パラメータ - Config


In [ ]:
resize_w = 256  # 画像のリサイズ幅を指定
resize_h = 256  # 画像のリサイズ高さを指定
channel = 3  # 画像のカラーチャンネルを指定

test_size_rate = 0.100 # データセットをTrain/Testに分割する際の、Test dataの比率（0.0〜1.0）

epochs = 200  # 学習回数を指定
n_batch = 8  # 一度にまとめて学習するデータ数。
             # 値が大きいと学習が安定し早く進むが、大きすぎると各データの特徴が平均化されるため逆に学習が進まない。
             # (GPU計算速度の関係で2のn乗を指定するのがおすすめ)

# 前準備 - Preprocess code

## Preprocess1. 画像処理関数定義 - Image Processing Functions

In [ ]:
import cv2
# 画像が大きいと計算が遅いため、リサイズ縮小
def resize(tmp_image):
    return cv2.resize(tmp_image , (resize_h, resize_w))

# 4次元配列化()　
def to_4d(tmp_image):
    return tmp_image.reshape(1, resize_h, resize_w, channel)   

# 256段階の色調を0.0~1.0にする
def normalize(tmp_image):
    return tmp_image / 255.0

def adjust(img, alpha=1.0, beta=0.0):
    # 積和演算を行う。
    dst = alpha * img + beta
    return np.clip(dst, 0.0, 255.0)

# 画像の前処理付きロード
def load_preprocessed_image(image_filepath):
    tmp_image = cv2.imread(image_filepath)
    tmp_image = resize(tmp_image)
    tmp_image = adjust(tmp_image, 1.3, 40)    
    tmp_image = normalize(tmp_image)
    tmp_image = to_4d(tmp_image)

    return tmp_image

## Preprocess2. 画像とラベルのロード - Load Dataset

In [ ]:
import pandas as pd
root_dir = "/kaggle/input/mj1-anomaly-images-detection-challenge/"
train_csv_filepath = root_dir + "train.csv"

# ファイルの読み込み
train_df = pd.read_csv(train_csv_filepath)

In [ ]:
import numpy as np
from keras.utils import np_utils

images = None
for fn in train_df['filename']:
    image_filepath = root_dir + 'train/' + fn
    tmp_image = load_preprocessed_image(image_filepath)
    if (images is None):
        images = tmp_image
    else:
        images = np.vstack((images, tmp_image))

anomaly_flags = np.array([flag for flag in train_df['anomaly']])
anomaly_flags = np_utils.to_categorical(anomaly_flags, 2)

## Preprocess3. Train/Test split

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size_rate, random_state=0)

for train_index, test_index in sss.split(images, anomaly_flags):
    X_train = images[train_index]
    y_train = anomaly_flags[train_index]
    X_test = images[test_index]
    y_test = anomaly_flags[test_index]

## Preprocess4. Over Sampling (不均衡な学習データ数を揃える処理）

In [ ]:
# over-samplingを試します。

tmp = pd.DataFrame(y_train[:, 1]).value_counts().values
print(tmp)
label_ok_num = tmp[0]
label_ng_num = tmp[1]

while(label_ok_num != label_ng_num):
    rand_index = np.random.randint(0, len(y_train))

    label_is_ng = (y_train[rand_index, 1] == 1.0)
    if label_is_ng:
        X_train = np.vstack((X_train, [X_train[rand_index]]))
        y_train = np.vstack((y_train, [y_train[rand_index]]))
        label_ng_num += 1
    print(label_ng_num, end='\r')

## Preprocess5. Augmantation (画像をランダムに変化させて、学習データにバリエーションをもたせる）

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=360)

datagen.fit(X_train)

# ■-- 転移学習(Transfer Learning)

転移学習は大まかにいうと、「すでに学習済みのオープンな高精度なモデルを流用して、簡単に自分の目的の学習モデルを作る方法」です。  

学習モデルには「1.入力データの特徴を捉える層」と「2.出力結果を推定する層」があり、  
1の層をそのままにして2の層を自分の目的に合うように差し替えることで転移学習を実現します.  

具体的には  
a. すでに学習済みのオープンな高精度なモデルを用意  
b. 学習しないようにパラメータ設定  
c. aの下に「2.出力結果を推定する層」を追加  
を行います。

---

Transfer learning is, roughly speaking, "a way to easily create a learning model for your own purposes by appropriating an open, highly accurate model that has already been trained.  

A learning model has two layers: 1. a layer that captures the characteristics of the input data, and 2. a layer that estimates the output results.  
Transfer learning can be achieved by leaving the first layer as it is and replacing the second layer with the one that suits your purpose.  

Specifically  
**a. Prepare an accurate open model that has already been trained.  
b. Set the parameters so that it does not learn.  
c. Add "2. Estimating the output result layer" under a. ** 


## ※注

学習済モデルのダウンロード  
> base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(  

の実行時にエラーになる場合があります。  

その場合は、このNoteBook の Internet 設定をオンにすることで、問題が解消されます。  
詳細は下記のリンクをご確認ください。  

https://stackoverflow.com/questions/47378542/kaggle-could-not-download-resnet50-pretrained-model

In [ ]:
import tensorflow as tf

# a. すでに学習済みのオープンな高精度なモデルを用意
# a. Prepare an accurate open model that has already been trained.  
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    weights='imagenet',
    include_top=False,
    pooling='avg'
)

# b. 学習しないようにパラメータ設定 
# b. Set the parameters so that it does not learn.  
base_model.trainable = False

# c. aの下に「2.出力結果を推定する層」を追加
# c. Add "2. Estimating the output result layer" under a. 
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(1024, activation='relu'), 
    tf.keras.layers.Dense(128, activation='relu'),        
    tf.keras.layers.Dense(2, activation='softmax')
])

model.summary()

> Total params: 3,701,186  
> Trainable params: 1,443,202  
> Non-trainable params: 2,257,984  

という結果になります。  
imagenetという汎用的な大規模画像データセットの学習済みモデルを使用しており、様々な画像の特徴が抜き出せるようになっています。  
転移学習をすることで、「1.入力データの特徴を捉える層」に必要だった2,257,984個のパラメータを学習する手間が省けました。

--- 


The results are shown above.

We have used imagenet, a general-purpose trained model of a large image dataset, which allows us to extract features from various images.  
By using transfer learning, we saved the time and effort of learning the 2,257,984 parameters required.

## 学習〜判定〜結果出力まで。

## 学習 - Learning Process

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=n_batch),
                    steps_per_epoch=len(X_train) / n_batch,
                    epochs=epochs,
                    validation_data=(X_test, y_test))

train_score = model.evaluate(X_train, y_train, verbose=0)
test_score = model.evaluate(X_test, y_test, verbose=0)
print('Train Loss:{0:.3f}'.format(train_score[0]))
print('Train accuracy:{0:.3}'.format(train_score[1]))
print('Test Loss:{0:.3f}'.format(test_score[0]))
print('Test accuracy:{0:.3}'.format(test_score[1]))

## 判定

In [ ]:
import glob
from pathlib import Path

test_images = None
test_filenames = None
for test_filepath in glob.glob('/kaggle/input/mj1-anomaly-images-detection-challenge/test/*.png'):
    tmp_image = load_preprocessed_image(test_filepath)
    if (test_images is None):
        test_images = tmp_image
        test_filenames = [Path(test_filepath).name]
    else:
        test_images = np.vstack((test_images, tmp_image))
        test_filenames.append(Path(test_filepath).name)

In [ ]:
result_predict = model.predict(test_images)
result_predict = np.argmax(result_predict, axis=1)
result_predict

## 結果をcsvで出力

In [ ]:
submit_filepath = "/kaggle/input/mj1-anomaly-images-detection-challenge/sample_submit.csv"
submit_df = pd.read_csv(submit_filepath, index_col=0)

for i, filename in enumerate(test_filenames):
    submit_df.loc[filename, 'Predicted'] = result_predict[i]
    
submit_df.to_csv('result_submit.csv')
submit_df[:20]

In [ ]:
for i, filename in enumerate(test_filenames):
    submit_df.loc[filename, 'Predicted'] = result_predict[i]
submit_df.to_csv('result_submit.csv')
print(submit_df)